# VT Graphs in Jupyter Notebook

In this notebook we will explore how to obtain attributes and relationship for different entities using VirusTotal API v3. Finally we can render all the relationships we have obtained using VTGraph.

## Import libraries

In [1]:
from msticpy.sectools.vtlookupv3 import VTLookupV3

import networkx as nx
import matplotlib.pyplot as plt
import os
import pandas as pd

import nest_asyncio
nest_asyncio.apply()

## Create Lookup instance

In [2]:
# Obtain key from env varaible
vt_key = os.environ["VT_API_KEY"]
# Instanciate vt_lookup object
vt_lookup = VTLookupV3(vt_key)

## Types of DataFrames

Our functions will return and accept two different types of DataFrames.

- **Attributes DataFrame**: Contains the properties of a entity. It has the columns:
    - id
    - type
    - ..properties
- **Relationships DataFrames**: Contains the relationship between two entities, and the type of relationship. It contains the columns:
    - source
    - target
    - source_type
    - target_type
    - relationship_type

In [3]:
FILE = 'ed01ebfbc9eb5bbea545af4d01bf5f1071661840480439c6e5babe8e080e41aa'

In [4]:
example_attribute_df = vt_lookup.lookup_ioc(observable=FILE, vt_type='file')
example_attribute_df

,last_submission_date,size,times_submitted,type_description,meaningful_name,first_submission_date,detections,scans,type
id,,,,,,,,,
ed01ebfbc9eb5bbea545af4d01bf5f1071661840480439c6e5babe8e080e41aa,1601421424,3514368,1481,Win32 EXE,diskpart.exe,1494574270,65,75,file


In [5]:
example_relationship_df = vt_lookup.lookup_ioc_relationships(
    observable=FILE, 
    vt_type='file', 
    relationship='execution_parents')
example_relationship_df

target_type  \
source                                             target                                                           
ed01ebfbc9eb5bbea545af4d01bf5f1071661840480439c... 018ac8f95d5e14b92011cdbfc8c48056ca4891161ed6bdd...        file   
                                                   02a7977d1faf7bfc93a4b678a049c9495ea663e7065aa5a...        file   
                                                   06b020a3fd3296bc4c7bf53307fe7b40638e7f445bdd43f...        file   
                                                   06c676bf8f5c6af99172c1cf63a84348628ae3f39df9e52...        file   
                                                   070f603e0443b1fae57425210fb3b27c2f77d8983cfefef...        file   
...                                                                                                           ...   
                                                   eca25aa20f7325d2b80e7701767c8d4c6ed96383bcfd364...        file   
                                                   ef57c7724d78e7571c642c0d49164d53edd4a9b62589329...        file   
                                                   f1aa23299987eed2173e83d26b6078232051f885586ebba...        file   
                                                   fbf74ee5d011dfb0d6c3357446ea3999ef62b088c553d66...        file   
                                                   ff6af3f113f61f823e422b7eb9e379495b81bdbb66a4e4e...        file   

                                                                                                      source_type  \
source                                             target                                                           
ed01ebfbc9eb5bbea545af4d01bf5f1071661840480439c... 018ac8f95d5e14b92011cdbfc8c48056ca4891161ed6bdd...        file   
                                                   02a7977d1faf7bfc93a4b678a049c9495ea663e7065aa5a...        file   
                                                   06b020a3fd3296bc4c7bf53307fe7b40638e7f445bdd43f...        file   
                                                   06c676bf8f5c6af99172c1cf63a84348628ae3f39df9e52...        file   
                                                   070f603e0443b1fae57425210fb3b27c2f77d8983cfefef...        file   
...                                                                                                           ...   
                                                   eca25aa20f7325d2b80e7701767c8d4c6ed96383bcfd364...        file   
                                                   ef57c7724d78e7571c642c0d49164d53edd4a9b62589329...        file   
                                                   f1aa23299987eed2173e83d26b6078232051f885586ebba...        file   
                                                   fbf74ee5d011dfb0d6c3357446ea3999ef62b088c553d66...        file   
                                                   ff6af3f113f61f823e422b7eb9e379495b81bdbb66a4e4e...        file   

                                                                                                       relationship_type  
source                                             target                                                                 
ed01ebfbc9eb5bbea545af4d01bf5f1071661840480439c... 018ac8f95d5e14b92011cdbfc8c48056ca4891161ed6bdd...  execution_parents  
                                                   02a7977d1faf7bfc93a4b678a049c9495ea663e7065aa5a...  execution_parents  
                                                   06b020a3fd3296bc4c7bf53307fe7b40638e7f445bdd43f...  execution_parents  
                                                   06c676bf8f5c6af99172c1cf63a84348628ae3f39df9e52...  execution_parents  
                                                   070f603e0443b1fae57425210fb3b27c2f77d8983cfefef...  execution_parents  
...                                                                                                                  ...  
                                                   eca25aa20f7325d2b80e770

### Obtaining result for multiple entities

The function `lookup_iocs` is able to obtain attributes for all the rows in a DataFrame. If no `observable_column` and `observable_type` parameters are specified, the function will obtain the attributes of all the entities that are in the column `target`, and will obtain their types from the `target_type` column.

This function is especially usefull when a user has obtained a set of relationships, and would like to obtain their attributes.

In [6]:
example_multiple_attribute_df = vt_lookup.lookup_iocs(example_relationship_df)
example_multiple_attribute_df

,size,last_submission_date,first_submission_date,times_submitted,type_description,meaningful_name,detections,scans,type
id,,,,,,,,,
018ac8f95d5e14b92011cdbfc8c48056ca4891161ed6bdd268770a5b56bb327f,3723264,1526215996,1495139411,6,Win32 EXE,8479206ff1a47362199ddabebb7358d2.virus,61,70,file
02a7977d1faf7bfc93a4b678a049c9495ea663e7065aa5a6caf0f69c5ff25dbd,9164800,1571387079,1570020111,4,Win32 EXE,=?UTF-8?B?572R5piT5bel5YW3566x56uv5ZCv5YqoLmV4...,53,72,file
06b020a3fd3296bc4c7bf53307fe7b40638e7f445bdd43fac1d04547a429fdaf,3991221,1588342161,1588342161,1,Win32 EXE,Tender.pdf.exe,42,74,file
06c676bf8f5c6af99172c1cf63a84348628ae3f39df9e523c42447e2045e00ff,4535704,1595479073,1595479073,1,Win32 EXE,car.exe,40,75,file
070f603e0443b1fae57425210fb3b27c2f77d8983cfefefb0ee185de572df33d,3723264,1601363298,1504687270,9,Win32 EXE,lhdfrgui.exe,64,75,file
...,...,...,...,...,...,...,...,...,...
eca25aa20f7325d2b80e7701767c8d4c6ed96383bcfd364148f7e4ce9de7886d,3482524,1600422027,1600422027,1,RAR,ultimate.rar,37,71,file
ef57c7724d78e7571c642c0d49164d53edd4a9b62589329ddb31da9e51ce9b68,5301112,1565622228,1565522903,2,Win32 EXE,EditCrypt.exe,45,69,file
f1aa23299987eed2173e83d26b6078232051f885586ebba35699143b83bc68ad,3723392,1563994865,1563994865,1,Win32 EXE,lhdfrgui.exe,64,72,file


Also, if we would like to obtain the relationships for a set of entities, we have the function `lookup_iocs_relationships`. Here also, if no `observable_column` and `observable_type` parameters are specified, the function will obtain the relationships of all the entities that are in the column `target`, and will obtain their types from the `target_type` column.

In [7]:
example_multiple_relationship_df = vt_lookup.lookup_iocs_relationships(example_relationship_df, 'contacted_domains')
example_multiple_relationship_df

target_type  \
source                                             target                                                          
018ac8f95d5e14b92011cdbfc8c48056ca4891161ed6bdd... www.iuqerfsodp9ifjaposdfjhgosurijfaewrwergwea.com      domain   
02a7977d1faf7bfc93a4b678a049c9495ea663e7065aa5a... fkksjobnn43.org                                        domain   
070f603e0443b1fae57425210fb3b27c2f77d8983cfefef... www.iuqerfsodp9ifjaposdfjhgosurijfaewrwergwea.com      domain   
1e06140672b73dfe337dfde7bc9dead5612bdbf4a8069be... www.iuqerfsodp9ifjaposdfjhgosurijfaewrwergwea.com      domain   
24d004a104d4d54034dbcffc2a4b19a11f39008a575aa61... www.iuqerfsodp9ifjaposdfjhgosurijfaewrwergwea.com      domain   
...                                                                                                          ...   
c8a8a17085c23f1af0d39dd20083b8edcf7e0701e308ac0... www.iuqerfsodp9ifjap1sdfjhgosurijfaewrwergwea.com      domain   
d2560b1043f7326569c3c9185fea1b5777053ea4e451e92... init-p01st.push.apple.com                              domain   
                                                   www.apple.com                                          domain   
e072cd048cd5ead7d22047412bf876a16442f1bb4deeb0b... www.ifferfsodp9ifjaposdfjhgosurijfaewrwergwea.com      domain   
f1aa23299987eed2173e83d26b6078232051f885586ebba... www.iuqerfsodp9ifjaposdfjhgosurijfaewrwergwea.com      domain   

                                                                                                     source_type  \
source                                             target                                                          
018ac8f95d5e14b92011cdbfc8c48056ca4891161ed6bdd... www.iuqerfsodp9ifjaposdfjhgosurijfaewrwergwea.com        file   
02a7977d1faf7bfc93a4b678a049c9495ea663e7065aa5a... fkksjobnn43.org                                          file   
070f603e0443b1fae57425210fb3b27c2f77d8983cfefef... www.iuqerfsodp9ifjaposdfjhgosurijfaewrwergwea.com        file   
1e06140672b73dfe337dfde7bc9dead5612bdbf4a8069be... www.iuqerfsodp9ifjaposdfjhgosurijfaewrwergwea.com        file   
24d004a104d4d54034dbcffc2a4b19a11f39008a575aa61... www.iuqerfsodp9ifjaposdfjhgosurijfaewrwergwea.com        file   
...                                                                                                          ...   
c8a8a17085c23f1af0d39dd20083b8edcf7e0701e308ac0... www.iuqerfsodp9ifjap1sdfjhgosurijfaewrwergwea.com        file   
d2560b1043f7326569c3c9185fea1b5777053ea4e451e92... init-p01st.push.apple.com                                file   
                                                   www.apple.com                                            file   
e072cd048cd5ead7d22047412bf876a16442f1bb4deeb0b... www.ifferfsodp9ifjaposdfjhgosurijfaewrwergwea.com        file   
f1aa23299987eed2173e83d26b6078232051f885586ebba... www.iuqerfsodp9ifjaposdfjhgosurijfaewrwergwea.com        file   

                                                                                                      relationship_type  
source                                             target                                                                
018ac8f95d5e14b92011cdbfc8c48056ca4891161ed6bdd... www.iuqerfsodp9ifjaposdfjhgosurijfaewrwergwea.com  contacted_domains  
02a7977d1faf7bfc93a4b678a049c9495ea663e7065aa5a... fkksjobnn43.org                                    contacted_domains  
070f603e0443b1fae57425210fb3b27c2f77d8983cfefef... www.iuqerfsodp9ifjaposdfjhgosurijfaewrwergwea.com  contacted_domains  
1e06140672b73dfe337dfde7bc9dead5612bdbf4a8069be... www.iuqerfsodp9ifjaposdfjhgosurijfaewrwergwea.com  contacted_domains  
24d004a104d4d54034dbcffc2a4b19a11f39008a575aa61... www.iuqerfsodp9ifjaposdfjhgosurijfaewrwergwea.com  contacted_domains  
...                                                                                                                 ...  
c8a8a17085c23f1af0d39dd20083b8edcf7e0701e308ac0... www.iuqerfsodp9ifjap1sdfjhgosurijfaewrwergwea.com  conta

## Integration with VTGraph

Once we have some DataFrames with the relationships, we are able to generate and visualize a VT Graph in our notebook. The function `create_vt_graph` accepts as input a **list of Relationship DataFrames**.

In [ ]:
graph_id = vt_lookup.create_vt_graph(
    relationship_dfs=[example_relationship_df],
    name="My first Jupyter Notebook Graph",
    private=True
)
graph_id

In [10]:
vt_lookup.render_vt_graph(
    graph_id = graph_id,
    width = 800,
    height = 500
)